In [ ]:
import tensorflow as tf
print(tf.__version__)


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import DenseFeatures

In [3]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [4]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [5]:
# Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)

# Drop un-used columns
df = df.drop(columns=['price'])

In [6]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels =  dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 10   #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [7]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [8]:
# numeric column
carat = feature_column.numeric_column('carat')
demo (carat)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.

[[2.1 ]
 [0.5 ]
 [0.9 ]
 [1.12]
 [0.41]
 [0.32]
 [0.3 ]
 [0.7 ]
 [1.63]
 [0.93]]


In [9]:
# bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [10]:
# Categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
    'color', ['E', 'I', 'J', 'D', 'H', 'G', 'F'])

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [11]:
# Embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
    'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[[ 0.22924729  0.31571868  0.3161583   0.00753899 -0.55990696  0.2623892 ]
 [ 0.4067011  -0.08763124 -0.06604492 -0.12525435 -0.42827097  0.22611696]
 [ 0.22924729  0.31571868  0.3161583   0.00753899 -0.55990696  0.2623892 ]
 [ 0.22924729  0.31571868  0.3161583   0.00753899 -0.55990696  0.2623892 ]
 [ 0.10197064  0.1848788  -0.35031492 -0.43938258  0.1095311  -0.49085385]
 [ 0.7271103   0.20606227  0.26175436  0.03214366 -0.03953018 -0.01279583]
 [ 0.22924729  0.31571868  0.3161583   0.00753899 -0.55990696  0.2623892 ]
 [ 0.4067011  -0.08763124 -0.06604492 -0.12525435 -0.42827097  0.22611696]
 [ 0.22924729  0.31571868  0.3161583   0.00753899 -0.55990696  0.2623892 ]
 [-0.25824854 -0.79951686  0.00209415 -0.5729346  -0.17265

In [12]:
# hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
    'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [13]:
# Cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [14]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))
 
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [15]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)


Epoch 1/10


1686/1686 [==============================] - 15s 7ms/step - loss: 0.2257 - accuracy: 0.9045 - val_loss: 0.1650 - val_accuracy: 0.9365
Epoch 2/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.1572 - accuracy: 0.9345 - val_loss: 0.1453 - val_accuracy: 0.9381
Epoch 3/10
1686/1686 [==============================] - 8s 5ms/step - loss: 0.1557 - accuracy: 0.9343 - val_loss: 0.1521 - val_accuracy: 0.9383
Epoch 4/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1515 - accuracy: 0.9363 - val_loss: 0.1419 - val_accuracy: 0.9379
Epoch 5/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1505 - accuracy: 0.9359 - val_loss: 0.1409 - val_accuracy: 0.9377
Epoch 6/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1473 - accuracy: 0.9366 - val_loss: 0.1603 - val_accuracy: 0.9249
Epoch 7/10
1686/1686 [==============================] - 8s 5ms/step - loss: 0.1456 - accuracy: 0.9369 - val_loss: 0.1391 - val_ac